# pre_processing

In [1]:
import pandas as pd
import numpy as np

In [2]:
import nbimporter
import feature_generation

Importing Jupyter notebook from feature_generation.ipynb
Importing Jupyter notebook from pre_processing.ipynb
Importing Jupyter notebook from feature_selection.ipynb


In [3]:
df = pd.read_csv('data/train.csv')

In [4]:
df.columns

Index(['id', 'titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio'],
      dtype='object')

In [3]:
features = feature_generation.get_features()
features_types = feature_generation.get_features_types()

In [4]:
def load_appended_dataset():
    """Carga ambos dataframes (train y test) y devuelve un solo dataframe que es el resultado de realizar un
    append entre ambos. El dataset de test esta al final."""

    train = pd.read_csv('./data/train.csv', index_col='id', parse_dates=['fecha'])
    test = pd.read_csv('./data/test.csv', index_col='id', parse_dates=['fecha'])
    df = train.append(test, sort=False)
   
    return df

In [5]:
def separate_train_test(df):
    """Separa df compuesto por sus primeras 240.000 filas de train.csv y sus 60.000 ultimas filas de test
    en los dos dataframes correspondientes."""
    
    train = df.head(240000)
    test = df.tail(60000).drop('precio', axis=1)
    return train, test

In [6]:
def load_datasets():
    """Carga ambos dataframes (train y test) y devuelve dos datasets en el formato:
    train_df, test_df"""

    df = load_appended_dataset()
    return separate_train_test(df)

In [4]:
def load_featured_appended_dataset(class_feature):
    """Carga el .csv con los features generados, y lo formatea para ahorrar memoria. Devuelve un unico
    dataframe cuyas primeras 240.000 filas son de train.csv y sus ultimas 60.000 de test."""
      
    df = load_appended_dataset()
    
    csv_names = feature_generation.get_csvs_names()
    features_types = feature_generation.get_features_types()
    
    for file in csv_names:
        if file=='class' and not class_feature:
            continue
        ruta = 'features/'+file+'.csv'
        add = pd.read_csv(ruta, index_col='id')
        df = pd.merge(df, add, how='left', on='id')
           
    for categoria in features_types.keys():
        if categoria=='class' and not class_feature:
            continue
        for tipo in features_types[categoria]:
            castear = features_types[categoria][tipo]
            if (tipo == 'bernoulli'):
                df[castear] = df[castear].astype('uint8')
            elif (tipo == 'float'):
                df[castear] = df[castear].astype('float')
            elif (tipo == 'uint8'):
                df[castear] = df[castear].astype('uint8')
            elif (tipo == 'uint16'):
                df[castear] = df[castear].astype('uint16')
            elif (tipo == 'uint32'):
                df[castear] = df[castear].astype('uint32')
    
    return df

In [5]:
def load_featured_datasets(class_feature=True):
    """Devuelve dos datasets en el formato:
    train_df, test_df
    Para los datasets con las features generadas."""
    
    df = load_featured_appended_dataset(class_feature)
    return separate_train_test(df)

# Ver el tema de dic 2016
En el TP1 vimos que habian muchas propiedades que eran de dic del 2016:


In [13]:
def aniomes(anio, mes):
    anio = str(anio)
    mes = str(mes)
    
    if len(mes)==1:
        mes = '0'+mes
    
    return int(anio+mes)

In [9]:
df = load_appended_dataset()

In [12]:
df['anio'] = df['fecha'].dt.year
df['mes'] = df['fecha'].dt.month
df['dia'] = df['fecha'].dt.day


In [14]:
df['aniomes'] = df.apply(lambda x: aniomes(x['anio'], x['mes']), axis=1)

In [17]:
df201612 = df.loc[df['aniomes'] == 201612]

In [28]:
df201612['precio'].std()

2354764.7388403495

In [31]:
# Vamos a chequear el desvio estandar de los precios para cada anio mes, para luego comparar con el std de
# diciembre del 2016.

for i in range(2,7):
    for j in range(1,13):
        anio = str(i)
        if len(str(j)) == 1:
            mes = '0'+str(j)
        else:
            mes = str(j)
        aniomes = int('201'+anio+mes)
        
        std = df.loc[df['aniomes'] == aniomes]['precio'].std()
        mean = df.loc[df['aniomes'] == aniomes]['precio'].mean()
        print(f"{aniomes} - std= {std} - mean= {mean}")
        

201201 - std= 1641045.1049234213 - mean= 1910287.3468586388
201202 - std= 1597102.887707227 - mean= 1817814.980406933
201203 - std= 1842065.0181309523 - mean= 1966355.1405258386
201204 - std= 1975269.7212173168 - mean= 2167271.0683403066
201205 - std= 1737103.5293206384 - mean= 1899123.956919763
201206 - std= 1725426.7793467424 - mean= 1930387.499343832
201207 - std= 1674625.858192183 - mean= 1827298.644196655
201208 - std= 1755932.3669465822 - mean= 2116829.444724251
201209 - std= 1846279.823949636 - mean= 2092779.5950731332
201210 - std= 1874879.4359990053 - mean= 2228156.793419804
201211 - std= 1877054.9610941103 - mean= 2188356.604477612
201212 - std= 1615653.3092376809 - mean= 1894107.711724138
201301 - std= 1741217.1513762006 - mean= 2032821.1996989462
201302 - std= 1824441.7826514642 - mean= 2158046.24910778
201303 - std= 1796898.6687960827 - mean= 2092787.9019607843
201304 - std= 1847035.049653021 - mean= 2164533.3367960635
201305 - std= 1807926.9052302793 - mean= 2119536.0345


In [30]:
# Vemos que si bien es creciente, no hay nada raro en el std del 201612.

<hr>

# Pre procesamiento de datasets externos
### Cotizacion USD

In [9]:
usd = pd.read_csv('external_datasets/usd_mxn_diario.csv')

#### funciones aux

In [10]:
def aniomes(x):
    anio=x[6:]
    mes=x[3:5]
    return anio+mes

def numeric(x):
    entero, fraccion = x.split(',')
    return int(entero) + int(fraccion)/10000

def varianza(x):
    x = x.rstrip('%')
    entero, fraccion = x.split(',')
    num = entero+"."+fraccion
    return float(num)

In [11]:
usd['aniomes'] = usd['Fecha'].map(aniomes)

In [12]:
valores = ['Último', 'Apertura', 'Máximo', 'Mínimo']
for valor in valores:
    usd[valor] = usd[valor].map(numeric)
    
usd['usd_varianza'] = usd['% var.'].map(varianza)
usd['daily_mean'] = usd.apply(lambda x: (x['Último'] + x['Apertura'])/2, axis=1)

In [13]:
usd = usd[['aniomes', 'usd_varianza', 'daily_mean']]

In [14]:
usd = usd.groupby('aniomes')['usd_varianza', 'daily_mean'].agg({'usd_varianza':'sum', 'daily_mean':'mean'})

In [15]:
usd.head()

,usd_varianza,daily_mean
aniomes,,
201201,-6.71,13.419316
201202,-1.50,12.789329
201203,-0.27,12.734559
201204,1.67,13.047955
201205,9.99,13.630126


In [16]:
usd.to_csv('data/usd_mxn_featured.csv')

# Volcanes

In [4]:
df = pd.read_csv('external_datasets/volcanes.csv')

In [5]:
df = df[['properties/OBJECTID', 'properties/LAT', 'properties/LONG_']]

In [6]:
df.columns = ['id', 'lat', 'long']

In [7]:
df.set_index('id', inplace=True)

In [29]:
df

,lat,long
id,,
19,19.030365,-97.268213
20,18.562021,-95.198504
21,19.493241,-102.251431
22,18.973674,-101.717852
23,19.308206,-110.806474
...,...,...
62,20.830000,-104.820000
59,28.500000,-113.750000
54,29.338829,-114.517599


In [8]:
df.to_csv('data/volcanes_featured.csv')

# Llenado de valores nulos en tipo de propiedad
Funcion que es llamada desde feature_generation

In [2]:
def buscar_tipo(x):
    lista = ['casas', 'departamento', 'casa', 'terrenos', 'terreno', 'departamentos',
             'apartamento', 'apartamentos', 'comercio', 'locales', 'condominio', 'residencia']
    palabras = x.lower().split(' ')
    categorias=[]
    for palabra in palabras:
        if palabra in lista:
            categorias.append(palabra)
    if len(categorias)==0:
        return ['Otros']
    elif len(categorias)>1:
        return casas_condominio(categorias)
    return categorias

def casas_condominio(lista):
    for palabra in lista:
        if palabra == 'condominio':
            return ['condominio']
    return lista

def translate_tiposdepropiedad(x):
    translate = {'casas':'Casa', 'departamento':'Apartamento', 'casa':'Casa', 'terrenos':'Terreno',
                 'terreno':'Terreno', 'departamentos':'Apartamento', 'apartamento':'Apartamento',
                 'apartamentos':'Apartamento', 'comercio':'Comercial', 'locales':'Comercial',
                 'condominio':'Casa en condominio', 'residencia':'Casa', 'Otros':'Otros'}
    return translate[x]

In [1]:
def fill_tipodepropiedad_nulls(nulos):
    nulos['tipodepropiedad_titulo'] = nulos['titulo'].map(buscar_tipo)
    nulos['tipodepropiedad_titulo'] = nulos['tipodepropiedad_titulo'].map(lambda x: translate_tiposdepropiedad(x[0]))
    
    return nulos.drop('tipodepropiedad', axis=1)